In [2]:
!pip install pandas
!pip install requests

Could not find platform independent libraries <prefix>



     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ---------------------------------------- 61.2/61.2 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/341.8 kB ? eta -:--:--
     -------------------------------------  337.9/341.8 kB 7.0 MB/s eta 0:00:01
     -------------------------------------- 341.8/341.8 kB 4.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   -- ------------------------------------- 0.7/10.5 MB 21.4 MB/s eta 0:00:01
   ----- ---------------------------------- 1.4/10.5 MB 14.7 MB/s eta 0:00:01
   ------- -------------------------------- 1.9/10.5 MB 13.7 MB/s eta 0:00:01
   --------- ------------------------------ 2.5/10.5 MB 13.2 MB/s eta 0:00:01
   ----------- ---------------------------- 3.0/10.5 MB 12.9 MB/s eta 0:00:01
   ------------- -------------------------- 3.6/10.5 MB 12.6 MB/s eta 0:00:01
   ---------------- ----------------------- 4.3/10.5 MB 12.9 MB/s eta 0:00:0

Could not find platform independent libraries <prefix>


In [25]:
import requests
import pandas as pd
import time
import json as js
import data_collection as dlol

In [6]:
URL = "https://americas.api.riotgames.com"
KEY = "RGAPI-0a0dfc77-970e-45ad-8ecf-63037fa1b2d6"

def get_summoner_id(summoner_name, api_key):
    url = f'{URL}/lol/summoner/v4/summoners/by-name/{summoner_name}?api_key={api_key}'
    response = requests.get(url)
    time.sleep(1)
    return response.json()

def get_match_list(api_key, summoner_id, start=0, count=10):
    url = f'{URL}/lol/match/v5/matches/by-puuid/{summoner_id}/ids?start={start}&count={count}&api_key={api_key}'
    response = requests.get(url)
    time.sleep(1)
    return response.json()

def get_match_data(match_id, api_key):
    url = f'{URL}/lol/match/v5/matches/{match_id}?api_key={api_key}'
    response = requests.get(url)
    time.sleep(1)
    return response.json()

def extract_match_info(match_data):
    game_duration = match_data['info']['gameDuration']
    game_mode = match_data['info']['gameMode']
    participants = match_data['info']['participants']
    
    participants_df = pd.DataFrame()
    
    for participant in participants:
        participant_info = {
            'championName': participant['championName'],
            'kills': participant['kills'],
            'death': participant['deaths'],
            'assists': participant['assists'],
            'totalDamageToChampions': participant['totalDamageDealtToChampions'],
            'goldEarned': participant['goldEarned'],
            'visioScore': participant['visionScore'],
            'role': participant['individualPosition'],
            'teamId': participant['teamId']
        }
        participants_df = pd.concat([participants_df, pd.DataFrame([participant_info])], ignore_index=True)
      
    
    return game_duration, game_mode, participants_df



In [15]:
raw_data = get_match_data('BR1_2839902431', KEY)
df_data = pd.json_normalize(raw_data)

raw_data['info']['participants']

[{'allInPings': 0,
  'assistMePings': 0,
  'assists': 1,
  'baitPings': 0,
  'baronKills': 0,
  'basicPings': 0,
  'bountyLevel': 0,
  'challenges': {'12AssistStreakCount': 0,
   'abilityUses': 328,
   'acesBefore15Minutes': 0,
   'alliedJungleMonsterKills': 2,
   'baronBuffGoldAdvantageOverThreshold': 1,
   'baronTakedowns': 1,
   'blastConeOppositeOpponentCount': 0,
   'bountyGold': 0,
   'buffsStolen': 1,
   'completeSupportQuestInTime': 0,
   'controlWardsPlaced': 0,
   'damagePerMinute': 650.5255010954779,
   'damageTakenOnTeamPercentage': 0.18399880623636528,
   'dancedWithRiftHerald': 0,
   'deathsByEnemyChamps': 3,
   'dodgeSkillShotsSmallWindow': 0,
   'doubleAces': 0,
   'dragonTakedowns': 1,
   'earliestBaron': 1406.8083619269337,
   'earliestDragonTakedown': 1548.2485632238527,
   'earlyLaningPhaseGoldExpAdvantage': 0,
   'effectiveHealAndShielding': 0,
   'elderDragonKillsWithOpposingSoul': 0,
   'elderDragonMultikills': 0,
   'enemyChampionImmobilizations': 45,
   'enemyJ

In [20]:


df = pd.json_normalize(raw_data['info']['participants'])
print(df.columns)

Index(['allInPings', 'assistMePings', 'assists', 'baitPings', 'baronKills',
       'basicPings', 'bountyLevel', 'champExperience', 'champLevel',
       'championId',
       ...
       'perks.statPerks.offense', 'perks.styles',
       'challenges.controlWardTimeCoverageInRiverOrEnemyHalf',
       'challenges.junglerKillsEarlyJungle',
       'challenges.killsOnLanersEarlyJungleAsJungler',
       'challenges.highestChampionDamage',
       'challenges.shortestTimeToAceFromFirstTakedown',
       'challenges.fastestLegendary',
       'challenges.fasterSupportQuestCompletion',
       'challenges.highestWardKills'],
      dtype='object', length=281)


In [22]:
participant_data = df['summonerName']
print(participant_data)

0    SingedEletroHIts
1     Sistema de Cota
2            shy wind
3          Puoiaiolam
4        PineapplePie
5          TyrannoCmg
6               Shárp
7            jgrm1010
8              kibao2
9      meninhaOGRANDE
Name: summonerName, dtype: object


In [24]:
df = df.set_index(participant_data)
df['championName']

summonerName
SingedEletroHIts         Gnar
Sistema de Cota        Lillia
shy wind                Yasuo
Puoiaiolam               Zeri
PineapplePie            Janna
TyrannoCmg          Gangplank
Shárp                   Ivern
jgrm1010                 Ryze
kibao2                 Darius
meninhaOGRANDE           Lulu
Name: championName, dtype: object